**Background**<br>
-The code below was designed to do analysis of trips of  **Indrive** an on demand mobile tax operator.<br>

**-The analysis will result in creation of:** <br>
1.Time in hours when trip was booked, <br>
2.Day of the week. <br> 
3.Number of trips taken by each customer.<br>
4.Time in days  taken between one trip and the other.<br> 
5.Grouping of customers in buckets based on number of trips they take.<br><br>

**NB: Visuals will be created in Power BI**<BR>
**The anlaysis was done in  AzureDataBricks** <BR><br>
**Import Libraries**

In [0]:
import pandas as pd
import numpy as np
from functools import reduce
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.window import Window
from pyspark.sql.functions import ntile
import pyspark.sql.functions as func
from pyspark.sql.functions import col
from pyspark.sql import functions as sf

**Extract data from spark table**

In [0]:

TripsData= spark.read.table("default.data_trips").dropna()
TripsData.limit(5).display()


iUserId,iTripId,tTripRequestDate,tDriverArrivedDate,tStartDate,tEndDate,Trip_Status
806906,940227,2022-02-05 06:55:07,2022-02-05 07:12:55,2022-02-05 07:13:06,2022-02-05 08:06:00,Finished
34161,940228,2022-02-05 07:19:37,2022-02-05 07:21:08,2022-02-05 07:22:37,2022-02-05 07:25:46,Finished
118258,940229,2022-02-05 07:26:03,2022-02-05 07:26:14,2022-02-05 07:31:53,2022-02-05 08:26:01,Finished
795458,940230,2022-02-05 07:26:31,2022-02-05 07:27:02,2022-02-05 07:27:14,2022-02-05 08:19:41,Finished
806953,940231,2022-02-05 07:30:49,2022-02-05 07:38:37,2022-02-05 07:46:09,2022-02-05 07:54:37,Finished


**Data Statistical Summary**

In [0]:
TripsData.summary().limit(5).display()

summary,iUserId,iTripId,tTripRequestDate,tDriverArrivedDate,tStartDate,tEndDate,Trip_Status
count,12262,12262,12262,12262,12262,12262,12262
mean,503835.2913302341,940994.0,null,null,null,null,null
stddev,351676.4402525227,3539.590159891396,null,null,null,null,null
min,100333,""",Yes,MAIMAI7QDSJKDR6WF,,0,Unsettelled,No,No,Ride,0,,,,,,0,,Regular,0,0,0,,,0,,,0,0,0,0,0,0,0,No,Driver,KMs,KMs,0,,No,No,""0000-00-00 00:00:00""",2022-01-11 17:36:10,0000-00-00 00:00:00,0000-00-00 00:00:00,0000-00-00 00:00:00,Canceled
25%,50475.0,937928.0,null,null,null,null,null


**Windows Time functions**<br>
-Use windows functions to generate time 

In [0]:

DriverArrival = (unix_timestamp('tDriverArrivedDate', "yyyy-MM-dd HH:mm:ss") - unix_timestamp('tTripRequestDate', "yyyy-MM-dd HH:mm:ss"))
trip_duration = (unix_timestamp('tEndDate', "yyyy-MM-dd HH:mm:ss") - unix_timestamp('tTripRequestDate', "yyyy-MM-dd HH:mm:ss"))
Limbo = (unix_timestamp('tEndDate', "yyyy-MM-dd HH:mm:ss") - unix_timestamp('tDriverArrivedDate', "yyyy-MM-dd HH:mm:ss"))


**Calculate the following from time**<br>
**1.Day of The Week**<br>
**2.Week Day**<br>
**3.Trip Request Hour**<br>
**4.Time Of Day**

In [0]:

Data_Trips = TripsData.withColumn("to_timestamp",to_timestamp(col("tTripRequestDate"), "yyyy-MM-dd HH:mm:ss"))\
                              .withColumn("DayofTheWeek",dayofweek("to_timestamp"))\
                              .withColumn("WeekDay", when (col("DayofTheWeek") == 1,"Sunday")
                              .when (col("DayofTheWeek") == 2,"Monday")
                              .when (col("DayofTheWeek") == 3,"Tuesday")
                              .when (col("DayofTheWeek") == 4,"Wednesday")
                              .when (col("DayofTheWeek") == 5,"Thursday")                       
                              .when (col("DayofTheWeek") == 6,"Friday")                       
                              .when (col("DayofTheWeek") == 7,"Saturday"))\
                              .withColumn("TripRequestHour",hour(col("to_timestamp")))\
                              .withColumn("TimeOfDay", when (col("TripRequestHour") == 6,"Early Morning")
                              .when (col("TripRequestHour") == 7,"Early Morning")
                              .when (col("TripRequestHour") == 8,"Early Morning")
                              .when (col("TripRequestHour") == 9,"Early Morning")
                              .when (col("TripRequestHour") == 10,"Mid Morning")                       
                              .when (col("TripRequestHour") == 11,"Mid Morning")                       
                              .when (col("TripRequestHour") == 12,"Mid Morning")                        
                              .when (col("TripRequestHour") == 13,"Lunch")                       
                              .when (col("TripRequestHour") == 14,"Lunch")
                              .when (col("TripRequestHour") == 15,"Lunch")                       
                              .when (col("TripRequestHour") == 16,"End Of Day")                       
                              .when (col("TripRequestHour") == 17,"End Of Day")                        
                              .when (col("TripRequestHour") == 18,"End Of Day")                        
                              .when (col("TripRequestHour") == 19,"Night Time")                        
                              .when (col("TripRequestHour") == 20,"Night Time")                        
                              .when (col("TripRequestHour") == 21,"Late Night")                        
                              .when (col("TripRequestHour") == 22,"Late Night")                        
                              .when (col("TripRequestHour") == 23,"Late Night")                        
                              .when (col("TripRequestHour") == 0,"Quite Hours")                       
                              .when (col("TripRequestHour") == 1,"Quite Hours")                        
                              .when (col("TripRequestHour") == 2,"Quite Hours")                        
                              .when (col("TripRequestHour") == 3,"Quite Hours")                       
                              .when (col("TripRequestHour") == 4,"Quite Hours")                       
                              .when (col("TripRequestHour") == 5,"Quite Hours")                                           
                              .otherwise("No Category Yet"))\
                              
Data_Trips.limit(5).display()    


iUserId,iTripId,tTripRequestDate,tDriverArrivedDate,tStartDate,tEndDate,Trip_Status,to_timestamp,DayofTheWeek,WeekDay,TripRequestHour,TimeOfDay
806906,940227,2022-02-05 06:55:07,2022-02-05 07:12:55,2022-02-05 07:13:06,2022-02-05 08:06:00,Finished,2022-02-05T06:55:07Z,7,Saturday,6,Early Morning
34161,940228,2022-02-05 07:19:37,2022-02-05 07:21:08,2022-02-05 07:22:37,2022-02-05 07:25:46,Finished,2022-02-05T07:19:37Z,7,Saturday,7,Early Morning
118258,940229,2022-02-05 07:26:03,2022-02-05 07:26:14,2022-02-05 07:31:53,2022-02-05 08:26:01,Finished,2022-02-05T07:26:03Z,7,Saturday,7,Early Morning
795458,940230,2022-02-05 07:26:31,2022-02-05 07:27:02,2022-02-05 07:27:14,2022-02-05 08:19:41,Finished,2022-02-05T07:26:31Z,7,Saturday,7,Early Morning
806953,940231,2022-02-05 07:30:49,2022-02-05 07:38:37,2022-02-05 07:46:09,2022-02-05 07:54:37,Finished,2022-02-05T07:30:49Z,7,Saturday,7,Early Morning


**Calculate Trips frequency**<br>
-Calculate the total number of trips taken by each customer 

In [0]:

Trips_Frequecny  = Data_Trips.groupBy("iUserId").count()
Trips_Frequecny.limit(5).display()


iUserId,count
813613,9
811178,83
814577,2
18130,22
18726,1


**Group customer**<br>
-Group the customers into buckets of 1 to 5 based on number of trips taken by each customer.<br> 
-Bucket 1 will be the least and 5 is the highest in trips taken by customers.

In [0]:

windowSpec = Window.orderBy("count")
Frequenttriptakers = Trips_Frequecny.withColumn("ntile",ntile(5).over(windowSpec)).orderBy(col('count').desc()).withColumnRenamed("count", "CountofTripsPerCustomer").withColumnRenamed("ntile", "Bucket_Group")
Frequenttriptakers.limit(5).display()


iUserId,CountofTripsPerCustomer,Bucket_Group
30881,178,5
809340,117,5
800939,110,5
39574,108,5
800942,101,5


**Join**<br>
-Trips is a result of a join of data trips df above and the frequence trips df above.<br>
-The outer join was used.

In [0]:

Trips = Data_Trips.join(Frequenttriptakers, on = ["iUserId"], how = "outer")
Trips.limit(5).display()


iUserId,iTripId,tTripRequestDate,tDriverArrivedDate,tStartDate,tEndDate,Trip_Status,to_timestamp,DayofTheWeek,WeekDay,TripRequestHour,TimeOfDay,CountofTripsPerCustomer,Bucket_Group
100333,946259,2022-03-08 13:40:08,2022-03-08 14:05:04,2022-03-08 14:05:04,2022-03-08 14:05:04,Canceled,2022-03-08T13:40:08Z,3,Tuesday,13,Lunch,1,2
101,940617,2022-02-07 07:44:53,2022-02-07 07:45:40,2022-02-07 07:48:56,2022-02-07 08:58:10,Finished,2022-02-07T07:44:53Z,2,Monday,7,Early Morning,40,5
101,940790,2022-02-07 19:43:36,2022-02-07 19:43:49,2022-02-07 20:41:10,2022-02-08 05:44:15,Finished,2022-02-07T19:43:36Z,2,Monday,19,Night Time,40,5
101,940838,2022-02-08 08:07:17,2022-02-08 08:07:27,2022-02-08 08:48:00,2022-02-08 18:20:35,Finished,2022-02-08T08:07:17Z,3,Tuesday,8,Early Morning,40,5
101,941001,2022-02-08 20:35:50,2022-02-08 20:36:00,2022-02-08 20:36:29,2022-02-08 23:12:53,Finished,2022-02-08T20:35:50Z,3,Tuesday,20,Night Time,40,5


**Calculating time lag of trips**<br>
-Calculating time when  previous trip was taken.

In [0]:

Trips_lag = Trips.withColumn('prevtrip',func.lag(Trips['to_timestamp']).over(Window.partitionBy("iUserId").orderBy("to_timestamp")))\

Trips_lag.limit(5).display()


iUserId,iTripId,tTripRequestDate,tDriverArrivedDate,tStartDate,tEndDate,Trip_Status,to_timestamp,DayofTheWeek,WeekDay,TripRequestHour,TimeOfDay,CountofTripsPerCustomer,Bucket_Group,prevtrip
100333,946259,2022-03-08 13:40:08,2022-03-08 14:05:04,2022-03-08 14:05:04,2022-03-08 14:05:04,Canceled,2022-03-08T13:40:08Z,3,Tuesday,13,Lunch,1,1,null
101,934978,2022-01-12 09:19:09,2022-01-12 09:19:30,2022-01-12 09:19:39,2022-01-12 10:32:05,Finished,2022-01-12T09:19:09Z,4,Wednesday,9,Early Morning,40,5,null
101,935117,2022-01-12 19:03:24,2022-01-12 19:03:33,2022-01-12 19:03:43,2022-01-12 21:36:20,Finished,2022-01-12T19:03:24Z,4,Wednesday,19,Night Time,40,5,2022-01-12T09:19:09Z
101,935222,2022-01-13 10:30:48,2022-01-13 10:31:05,2022-01-13 11:44:02,2022-01-13 12:55:12,Finished,2022-01-13T10:30:48Z,5,Thursday,10,Mid Morning,40,5,2022-01-12T19:03:24Z
101,935469,2022-01-14 11:03:42,2022-01-14 11:04:06,2022-01-14 12:35:56,2022-01-14 14:49:40,Finished,2022-01-14T11:03:42Z,6,Friday,11,Mid Morning,40,5,2022-01-13T10:30:48Z


**Calculating time lag of trips in seconds**<br>
-There is need to calculate the difference between trips in seconds so as to do a more granular analysis of trips time lags.

In [0]:

TimeInBetween_Trips = Trips_lag.withColumn(('Trip_Time_Difference(Seconds)'),((Trips_lag['to_timestamp'].cast("long")-Trips_lag['prevtrip'].cast("long"))))
TimeInBetween_Trips.limit(5).display()


iUserId,iTripId,tTripRequestDate,tDriverArrivedDate,tStartDate,tEndDate,Trip_Status,to_timestamp,DayofTheWeek,WeekDay,TripRequestHour,TimeOfDay,CountofTripsPerCustomer,Bucket_Group,prevtrip,Trip_Time_Difference(Seconds)
100333,946259,2022-03-08 13:40:08,2022-03-08 14:05:04,2022-03-08 14:05:04,2022-03-08 14:05:04,Canceled,2022-03-08T13:40:08Z,3,Tuesday,13,Lunch,1,2,null,null
101,934978,2022-01-12 09:19:09,2022-01-12 09:19:30,2022-01-12 09:19:39,2022-01-12 10:32:05,Finished,2022-01-12T09:19:09Z,4,Wednesday,9,Early Morning,40,5,null,null
101,935117,2022-01-12 19:03:24,2022-01-12 19:03:33,2022-01-12 19:03:43,2022-01-12 21:36:20,Finished,2022-01-12T19:03:24Z,4,Wednesday,19,Night Time,40,5,2022-01-12T09:19:09Z,35055
101,935222,2022-01-13 10:30:48,2022-01-13 10:31:05,2022-01-13 11:44:02,2022-01-13 12:55:12,Finished,2022-01-13T10:30:48Z,5,Thursday,10,Mid Morning,40,5,2022-01-12T19:03:24Z,55644
101,935469,2022-01-14 11:03:42,2022-01-14 11:04:06,2022-01-14 12:35:56,2022-01-14 14:49:40,Finished,2022-01-14T11:03:42Z,6,Friday,11,Mid Morning,40,5,2022-01-13T10:30:48Z,88374


**Calcuting time in average days from one trip to another**<br>
-This is very critical because  it enables us to know how many days at average one customer takes another trip from the previous trip.

In [0]:

Trips =    TimeInBetween_Trips.withColumn("Trip_Time_Difference(Hours)",col("Trip_Time_Difference(Seconds)")/ 3600)\
                                                .withColumn("Trip_Time_Difference(Days)",col("Trip_Time_Difference(Hours)")/ 24)\
                                                .withColumn("User_Average_Time_Between_Trips(Days)", avg("Trip_Time_Difference(Days)").over(Window.partitionBy("iUserId")))\
                                                .withColumn("Bucket_Group_Average_Time_Between_Trips(Days)", avg("Trip_Time_Difference(Days)").over(Window.partitionBy("Bucket_Group")))
                                                
Trips.limit(5).display()


iUserId,iTripId,tTripRequestDate,tDriverArrivedDate,tStartDate,tEndDate,Trip_Status,to_timestamp,DayofTheWeek,WeekDay,TripRequestHour,TimeOfDay,CountofTripsPerCustomer,Bucket_Group,prevtrip,Trip_Time_Difference(Seconds),Trip_Time_Difference(Hours),Trip_Time_Difference(Days),User_Average_Time_Between_Trips(Days),Bucket_Group_Average_Time_Between_Trips(Days)
10195,936906,2022-01-21 13:29:45,2022-01-21 13:40:02,2022-01-21 13:40:02,2022-01-21 13:40:02,Canceled,2022-01-21T13:29:45Z,6,Friday,13,Lunch,1,1,null,null,null,null,null,null
10460,935879,2022-01-16 00:25:06,2022-01-16 00:25:56,2022-01-16 00:25:56,2022-01-16 00:25:56,Canceled,2022-01-16T00:25:06Z,1,Sunday,0,Quite Hours,1,1,null,null,null,null,null,null
107477,935752,2022-01-15 12:31:51,2022-01-15 12:51:56,2022-01-15 12:52:49,2022-01-15 13:27:35,Finished,2022-01-15T12:31:51Z,7,Saturday,12,Mid Morning,1,1,null,null,null,null,null,null
107772,937991,2022-01-26 17:33:44,2022-01-26 17:47:50,2022-01-26 17:56:24,2022-01-26 18:15:52,Finished,2022-01-26T17:33:44Z,4,Wednesday,17,End Of Day,1,1,null,null,null,null,null,null
10933,938283,2022-01-28 06:19:35,2022-01-28 06:30:24,2022-01-28 06:30:24,2022-01-28 06:30:24,Canceled,2022-01-28T06:19:35Z,6,Friday,6,Early Morning,1,1,null,null,null,null,null,null


**Sample**<br>
-A filter of a very active customer

In [0]:

Sample = Trips.filter((Trips.iUserId == "30881")).limit(5).display()


iUserId,iTripId,tTripRequestDate,tDriverArrivedDate,tStartDate,tEndDate,Trip_Status,to_timestamp,DayofTheWeek,WeekDay,TripRequestHour,TimeOfDay,CountofTripsPerCustomer,Bucket_Group,prevtrip,Trip_Time_Difference(Seconds),Trip_Time_Difference(Hours),Trip_Time_Difference(Days),User_Average_Time_Between_Trips(Days),Bucket_Group_Average_Time_Between_Trips(Days)
30881,934881,2022-01-11 18:43:10,2022-01-11 18:43:25,2022-01-11 18:43:42,2022-01-11 22:06:34,Finished,2022-01-11T18:43:10Z,3,Tuesday,18,End Of Day,178,5,null,null,null,null,0.3365615845365139,2.395852757440276
30881,935000,2022-01-12 11:12:19,2022-01-12 11:12:25,2022-01-12 11:12:25,2022-01-12 11:12:25,Canceled,2022-01-12T11:12:19Z,4,Wednesday,11,Mid Morning,178,5,2022-01-11T18:43:10Z,59349,16.485833333333332,0.6869097222222221,0.3365615845365139,2.395852757440276
30881,935001,2022-01-12 11:14:17,2022-01-12 11:14:23,2022-01-12 11:14:23,2022-01-12 11:14:23,Canceled,2022-01-12T11:14:17Z,4,Wednesday,11,Mid Morning,178,5,2022-01-12T11:12:19Z,118,0.03277777777777778,0.001365740740740741,0.3365615845365139,2.395852757440276
30881,935002,2022-01-12 11:16:28,2022-01-12 11:17:24,2022-01-12 11:17:47,2022-01-12 14:45:19,Finished,2022-01-12T11:16:28Z,4,Wednesday,11,Mid Morning,178,5,2022-01-12T11:14:17Z,131,0.03638888888888889,0.0015162037037037036,0.3365615845365139,2.395852757440276
30881,935048,2022-01-12 15:44:47,2022-01-12 15:45:01,2022-01-12 15:45:23,2022-01-12 17:39:25,Finished,2022-01-12T15:44:47Z,4,Wednesday,15,Lunch,178,5,2022-01-12T11:16:28Z,16099,4.4719444444444445,0.18633101851851852,0.3365615845365139,2.395852757440276
